## Iterative Adv Training

### For iter in Range(20):
-       Step 1: Generate training adv examples to 1) overwrite shot_adv.png 2) create coco_perturb_train.json
-       Step 2: Mix downsampled normal training data with perturbed training data (4:1) and create coco_adv_mix.json
-       Step 3: Evaluate training Acc on all clean+adv examples (9 metrics)
-       Step 4: Gerenate testing adv examples .... 

In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import pickle

In [2]:
import os
os.chdir('..')

In [3]:
!pwd

/home/l/liny/yizhe/code/detectron2-1


In [ ]:

for i in range(20):
    if i == 0:
        weight_last_epoch = 'output/rcnn_bet365/model_final.pth'
    else:
        weight_last_epoch = 'output/adv_%s/model_final.pth'%(str(i - 1))
    output_this_epoch = 'output/adv_%s'%(str(i))
    weight_this_epoch = output_this_epoch + '/model_final.pth'
    
    '''generate training, testing adv examples'''
    !python run_DAG.py --test False --cfg-path configs/faster_rcnn_bet365.yaml \
                                               --weights-path {weight_last_epoch} \
                                               --results-save-path coco_instances_ignore_train.json 

    !python run_DAG.py --test True --cfg-path configs/faster_rcnn_bet365.yaml \
                                              --weights-path {weight_last_epoch} \
                                              --results-save-path coco_instances_ignore_test.json 
        
    #mix normal with training
    !python detectron2_1/mix_normal_adv.py --normal-path data/benign_data/coco_train.json \
                                           --adv-path data/benign_data/coco_perturbgt_train.json \
                                           --save-path data/benign_data/coco_adv_mix.json 

    #retrain 
    !python train_net.py --config-file configs/faster_rcnn_adv.yaml \
                                       OUTPUT_DIR {output_this_epoch} \
                                       MODEL.WEIGHTS {weight_last_epoch} \
    
    # evaluate training acc
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv_eval_train.yaml \
                                                   MODEL.WEIGHTS {weight_this_epoch} \
                                                   OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_adv_mix.json' # here the ground-truth is for the original-sized image
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' # here the prediction is for the resized image
    coco_gt = COCO(gt_coco_path)
    coco_dt = coco_gt.loadRes(results_coco_path)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    with open(output_this_epoch + '/all_trainacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [1]
    with open(output_this_epoch + '/input_trainacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
        
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [2]
    with open(output_this_epoch + '/logo_trainacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
        
    # evaluate testing acc
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv_eval_test.yaml \
                                     MODEL.WEIGHTS {weight_this_epoch} \
                                     OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_perturbgt_test.json' 
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' 
    coco_gt = COCO(gt_coco_path)
    coco_dt = coco_gt.loadRes(results_coco_path)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    with open(output_this_epoch + '/all_testacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [1]
    with open(output_this_epoch + '/input_testacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [2]
    with open(output_this_epoch + '/logo_testacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
    
        

Done with attack. Total Iterations 149
464it [1:25:48, 11.07s/it][464/1579] Attacking data/benign_data/benign_database/buxone.ru/shot.png ...
Done with attack. Total Iterations 149
465it [1:25:58, 10.96s/it][465/1579] Attacking data/benign_data/benign_database/adam4adam.tv/shot.png ...
Done with attack. Total Iterations 149
466it [1:26:10, 11.02s/it][466/1579] Attacking data/benign_data/benign_database/studentenwerk-dresden.de/shot.png ...
Done with attack. Total Iterations 149
467it [1:26:23, 11.67s/it][467/1579] Attacking data/benign_data/benign_database/ru.is/shot.png ...
Done with attack. Total Iterations 149
468it [1:26:35, 11.91s/it][468/1579] Attacking data/benign_data/benign_database/handtec.co.uk/shot.png ...
Done with attack. Total Iterations 149
469it [1:26:47, 11.92s/it][469/1579] Attacking data/benign_data/benign_database/wize.life/shot.png ...
Done with attack. Total Iterations 149
470it [1:26:59, 11.93s/it][470/1579] Attacking data/benign_data/benign_database/floorplanne

In [ ]:
with open('./output/adv_1/logo_testacc.pkl', 'rb') as f:
    logo_testacc = pickle.load(f)
    
with open('./output/adv_1/input_testacc.pkl', 'rb') as f:
    input_testacc = pickle.load(f)
    
with open('./output/adv_1/all_testacc.pkl', 'rb') as f:
    all_testacc = pickle.load(f)

In [ ]:
with open('./output/adv_1/logo_trainacc.pkl', 'rb') as f:
    logo_trainacc = pickle.load(f)
    
with open('./output/adv_1/input_trainacc.pkl', 'rb') as f:
    input_trainacc = pickle.load(f)
    
with open('./output/adv_1/all_trainacc.pkl', 'rb') as f:
    all_trainacc = pickle.load(f)

In [ ]:
logo_testacc.evaluate()
logo_testacc.accumulate()
logo_testacc.summarize()